In [1]:
import pandas as pd
import numpy as np

#pd.set_option('display.max_columns', 200)
#pd.set_option('display.max_rows', None)

In [2]:
df_1 = pd.read_csv("train_transaction.csv")
#df_2 = pd.read_csv("train_identity.csv")
#df_3 = pd.read_csv('test_transaction.csv')

In [ ]:
# EDA

In [3]:
df_1

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
for col_name in df_1.columns :
    if df_1[col_name].isnull().sum() /len(df_1[col_name])  < 0.5 :
        print(f"col_name {col_name} empty {df_1[col_name].isnull().sum()}")
        

col_name TransactionID empty 0
col_name isFraud empty 0
col_name TransactionDT empty 0
col_name TransactionAmt empty 0
col_name ProductCD empty 0
col_name card1 empty 0
col_name card2 empty 8933
col_name card3 empty 1565
col_name card4 empty 1577
col_name card5 empty 4259
col_name card6 empty 1571
col_name addr1 empty 65706
col_name addr2 empty 65706
col_name P_emaildomain empty 94456
col_name C1 empty 0
col_name C2 empty 0
col_name C3 empty 0
col_name C4 empty 0
col_name C5 empty 0
col_name C6 empty 0
col_name C7 empty 0
col_name C8 empty 0
col_name C9 empty 0
col_name C10 empty 0
col_name C11 empty 0
col_name C12 empty 0
col_name C13 empty 0
col_name C14 empty 0
col_name D1 empty 1269
col_name D2 empty 280797
col_name D3 empty 262878
col_name D4 empty 168922
col_name D10 empty 76022
col_name D11 empty 279287
col_name D15 empty 89113
col_name M1 empty 271100
col_name M2 empty 271100
col_name M3 empty 271100
col_name M4 empty 281444
col_name M6 empty 169360
col_name V1 empty 279287
col

In [ ]:
# FILLING NULL/EMPTY VALUES

In [6]:
# Iterate through each column in the DataFrame
for column in df_1.columns:
    # Check if the column contains numerical data
    if pd.api.types.is_numeric_dtype(df_1[column]):
        # Fill null or nan values with the mean of the column
        df_1[column] = df_1[column].fillna(df_1[column].mean())
    else:
        # Fill null or nan values with the most frequent element (mode) of the column
        df_1[column] = df_1[column].fillna(df_1[column].mode()[0])

In [7]:
df_1.isnull().sum()

TransactionID     0
isFraud           0
TransactionDT     0
TransactionAmt    0
ProductCD         0
                 ..
V335              0
V336              0
V337              0
V338              0
V339              0
Length: 394, dtype: int64

In [8]:
df_1.drop('TransactionID', axis=1, inplace= True)

In [12]:
# As seen number of fraud transactions are very low compared to legal
# transactions. In order to better train the model. we re-sample among
# legal transactions and increase the proportion of fraud transactions. 
df_1['isFraud'].value_counts()

isFraud
0    569877
1     20663
Name: count, dtype: int64

In [24]:
df_l = df_1 [df_1['isFraud'] == 0]
df_f = df_1 [df_1['isFraud'] == 1]
dfs = df_l.sample(len(df_f))
dff = pd.concat([df_f, dfs])
dff = dff.sample(frac=1, random_state=42)  # 'frac=1' shuffles the entire DataFrame
df = dff.reset_index(drop=True)
df

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,0,8361627,117.000,W,7676,512.0,150.0,visa,226.0,debit,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
1,1,13458191,994.000,W,16132,111.0,150.0,visa,226.0,debit,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
2,1,3146178,115.000,W,5964,268.0,150.0,visa,166.0,debit,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
3,0,14667251,29.000,W,10616,583.0,150.0,visa,226.0,credit,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
4,1,9580231,13.655,C,8121,555.0,185.0,visa,166.0,debit,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41321,1,4756811,450.000,H,15511,562.0,150.0,mastercard,197.0,credit,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
41322,1,8218403,71.054,C,13832,375.0,185.0,mastercard,224.0,debit,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
41323,0,8815384,51.000,W,12501,490.0,150.0,visa,226.0,debit,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
41324,1,759063,57.017,C,9635,405.0,185.0,visa,226.0,credit,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882


In [27]:
y = df['isFraud']
x = df.drop('isFraud',axis=1, inplace = False)
x = pd.get_dummies(x,drop_first=True)

In [28]:
x

,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,...,M1_T,M2_T,M3_T,M4_M1,M4_M2,M5_T,M6_T,M7_T,M8_T,M9_T
0,8361627,117.000,7676,512.0,150.0,226.0,299.000000,87.00000,118.50218,231.855423,...,True,True,True,False,False,True,True,False,False,True
1,13458191,994.000,16132,111.0,150.0,226.0,299.000000,87.00000,6.00000,231.855423,...,True,True,True,False,False,False,True,False,False,True
2,3146178,115.000,5964,268.0,150.0,166.0,181.000000,87.00000,118.50218,231.855423,...,True,True,True,False,False,True,False,False,False,True
3,14667251,29.000,10616,583.0,150.0,226.0,325.000000,87.00000,8.00000,231.855423,...,True,True,True,True,False,True,False,False,False,True
4,9580231,13.655,8121,555.0,185.0,166.0,290.733794,86.80063,118.50218,231.855423,...,True,True,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41321,4756811,450.000,15511,562.0,150.0,197.0,472.000000,87.00000,118.50218,231.855423,...,True,True,True,False,False,False,False,False,False,True
41322,8218403,71.054,13832,375.0,185.0,224.0,290.733794,86.80063,118.50218,342.000000,...,True,True,True,False,True,False,False,False,False,True
41323,8815384,51.000,12501,490.0,150.0,226.0,204.000000,87.00000,118.50218,231.855423,...,True,True,True,False,False,True,False,False,False,True
41324,759063,57.017,9635,405.0,185.0,226.0,161.000000,60.00000,118.50218,1.000000,...,True,True,True,False,True,False,False,False,False,True


In [29]:
def algo_test(x,y):
#def algo_test(x_train,x_test,y_train,y_test):
    
    import numpy as np

    from scipy import stats
    import matplotlib.pyplot as plt
    %matplotlib inline 
    import seaborn as sns


    from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
    from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
    from sklearn.tree import ExtraTreeClassifier
    from sklearn.ensemble import GradientBoostingRegressor
    from xgboost import XGBRegressor 

    from sklearn.model_selection import train_test_split

    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    from sklearn.naive_bayes import GaussianNB
    from sklearn.naive_bayes import BernoulliNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import ExtraTreesClassifier
    from xgboost import XGBClassifier

    import warnings
    warnings.filterwarnings('ignore')
#plt.style.use('')

    
    
    G = GaussianNB()
    B = BernoulliNB()
    #K = KNeighborsClassifier()
    L = LogisticRegression()
    D = DecisionTreeClassifier()
    RF = RandomForestClassifier()
    ETC = ExtraTreesClassifier()
    XG = XGBClassifier()

    
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state = 13)
    
    algos = [G,B,L,D, RF, ETC, XG]
    #algos = [G,B,L,D,  RF, ETC]
    
    #algo_names = [ "Gaussian", "Bernoulli", "K-Neighbors", "Logistic Regression", "Decision Tree", "Random Forest", "ExtraTreeClassifier" ]
    algo_names = [ "Gaussian", "Bernoulli","Logistic Regression", "Decision Tree", \
                  "Random Forest", "ExtraTreeClassifier","XGBClassifier" ]
    
    ASC = [];
    
    result =  result = pd.DataFrame(columns =['Accuracy_Score'], index = algo_names)
    
    i = 0;
    for algo in algos :
        
        algo.fit(x_train,y_train)
        #print (accuracy_score(  y_test, algo.predict(x_test) ))
        ASC.append(  accuracy_score(  y_test, algo.predict(x_test) ) )
        
        print( "Confusion matrix: ",format(algo_names[i]) )
        i = i+1;
        print ( confusion_matrix(algo.predict(x_test),y_test) )
        #print (classification_report(y_test,algo.predict(x_test) ))
        
        #algo.fit(x,y)
        #print (accuracy_score(  y, algo.predict(x) ))
        #ASC.append(  accuracy_score(  y, algo.predict(x) ) )
    
    
    result.Accuracy_Score = ASC;

    return result.sort_values('Accuracy_Score', ascending=False)

In [30]:
algo_test(x,y)

Confusion matrix:  Gaussian
[[  91  139]
 [3996 4040]]
Confusion matrix:  Bernoulli
[[3117 1855]
 [ 970 2324]]
Confusion matrix:  Logistic Regression
[[3144 2008]
 [ 943 2171]]
Confusion matrix:  Decision Tree
[[3166  832]
 [ 921 3347]]
Confusion matrix:  Random Forest
[[3557  722]
 [ 530 3457]]
Confusion matrix:  ExtraTreeClassifier
[[3569  688]
 [ 518 3491]]
Confusion matrix:  XGBClassifier
[[3632  703]
 [ 455 3476]]


,Accuracy_Score
XGBClassifier,0.859908
ExtraTreeClassifier,0.854101
Random Forest,0.848536
Decision Tree,0.787926
Bernoulli,0.658239
Logistic Regression,0.642995
Gaussian,0.499758
